# Задание 3

#### CounterGetter (0.1 балл)
Реализуйте класс CounterGetter, который на вход принимает произвольное количество именованых аргументов и считает количество обращений к полям и методам класса во всех экземплярах.

In [1]:
class CounterGetter:
    _count = 0

    def __init__(self, **kwargs):
        for name, value in kwargs.items():
            object.__setattr__(self, name, value)

    def __getattribute__(self, item):
        value = object.__getattribute__(self, item)
        CounterGetter._count += 1
        return value

    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)
        CounterGetter._count += 1

    @staticmethod
    def get_count():
        return CounterGetter._count
    


In [2]:
c = CounterGetter(a=1, x=2)
print(c.a, c.x)
try:
    print(c.b)
except AttributeError:
    pass
c.b = 5
print(c.get_count())

1 2
4


#### Vector (0.4 балла)

Реализуйте класс вектор, который должен обладать следующими свойствами:
* Над экземплярами выполняются арифметические операции (+-\*/). Операции могут выполняться как с числами, так и с векторами. Если второй операнд вектор, то верните их скалярное произведение, если число, выполните поэлементное действие.
* Реализуйте доступ к элементам вектора по индексам:

<code>vec = Vector(1, 2, 3)

vec[0]</code>
* Реализуйте умножение вектора на матрицу (Матрица задается двумерным списком).
* Поддерживайте методы push_back, pop_back, insert.
* Реализуйте поддержку функции len

P. S. Запрещается использовать numpy и другие библиотеки по работе с векторами. Рекомендуется представлять элементы вектора как список.

In [3]:
import numbers


class Vector(object):
    def __init__(self, *args):
        if len(args) == 0:
            self._storage = []
        elif len(args) == 1:
            self._storage = list(args[0])
        else:
            self._storage = list(args)

    def __len__(self):
        return len(self._storage)

    def __getitem__(self, item):
        if type(item) == slice:
            return NotImplemented
        else:
            return self._storage[item]

    def __setitem__(self, key, value):
        if type(key) == slice:
            return NotImplemented
        else:
            self._storage[key] = value

    def __repr__(self):
        return f'Vector({", ".join(map(repr, self._storage))})'

    def __str__(self):
        return f'Vector({", ".join(map(str, self._storage))})'

    def __add__(self, other):
        if type(other) != Vector:
            return NotImplemented

        self._check_vector_len(other)
        return Vector(map(lambda x, y: x + y, self._storage, other._storage))

    def __sub__(self, other):
        if type(other) != Vector:
            return NotImplemented

        self._check_vector_len(other)
        return Vector(map(lambda x, y: x - y, self._storage, other._storage))

    def __mul__(self, other):
        if type(other) == Vector:
            self._check_vector_len(other)
            return sum(map(lambda x, y: x * y, self._storage, other._storage))
        elif isinstance(other, numbers.Number):
            return Vector(map(lambda x: x * other, self._storage))
        else:
            return NotImplemented

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        if not isinstance(other, numbers.Number):
            return NotImplemented
        return Vector(map(lambda x: x / other, self._storage))

    def _check_vector_len(self, other):
        if len(other) != len(self):
            raise ValueError("operand Vectors aren't length equal")

    def push_back(self, value):
        self._storage.append(value)

    def pop_back(self):
        value = self._storage[-1]
        del self._storage[-1]
        return value

    def insert(self, pos, value):
        self._storage.insert(pos, value)

    def matrix_mult(self, matrix):
        if type(matrix) != list or any(map(lambda x: type(x) != list, matrix)):
            raise TypeError('Matrix must be a list of lists')
        new_len = len(matrix[0])
        if len(matrix) != len(self):
            raise ValueError('Matrix height and vector length must be equal')
        if any(map(lambda x: len(x) != new_len, matrix)):
            raise ValueError('All rows in matrix must have the same length')

        res = Vector()
        for i in range(new_len):
            res.push_back(sum(map(lambda x, y: x * matrix[y][i],
                                  self._storage,
                                  range(len(self)))))
        return res
    


In [4]:
a = Vector(1, 1.5, -5)
b = Vector(-4, 0, 3.14)
c = Vector(1, 2)

In [5]:
a + b

Vector(-3, 1.5, -1.8599999999999999)

In [6]:
a + c

ValueError: operand Vectors aren't length equal

In [7]:
a * b

-19.700000000000003

In [8]:
a * 2

Vector(2, 3.0, -10)

In [9]:
2 * a

Vector(2, 3.0, -10)

In [10]:
a / 2

Vector(0.5, 0.75, -2.5)

In [11]:
len(a), len(b), len(c)

(3, 3, 2)

In [12]:
a.matrix_mult([[1, 4, 7], [2, 5, 8], [3, 6, 9]])

Vector(-11.0, -18.5, -26.0)

#### Table (0.5 баллов)

Реализуйте класс для работы с таблицами. Класс должен обладать следующей функциональностью:

* Таблица может задаваться как двумерным списком, так и читаться из файла. Если читается из файла, то должен быть указан разделитель.

* Обладать возможностью задания названия колонок (по номеру, если не задано). Если читается из файла и задан параметр наличия именованных колонок, то первая строчка файла отвечает за названия колонок.

* Реализуйте доступ к колонкам по названю и по индекс:

<code>t = Table()

t["index"]</code>

* Реализуйте функции head и tail, которые показывают заданное число строк с начала и с конца соответственно.
* Реадизуйте методы unique и count, которые выводят все уникальные элементы для таблицы или для заданного столбца:

<code>t["sex"].unique()

t.unique()</code>

In [25]:
class Table:
    class _Column:
        def __init__(self, iterable, name=None):
            self._storage = list(iterable)
            self._name = name

        @property
        def name(self):
            return self._name

        @name.setter
        def name(self, value):
            if type(value) != str and value is not None:
                raise TypeError("columns name must be 'str' or None")
            self._name = value

        def __len__(self):
            return len(self._storage)

        def __getitem__(self, item):
            return self._storage[item]

        def __setitem__(self, key, value):
            self._storage[key] = value

        def unique(self):
            return set(self._storage)

        def count(self):
            return len(self.unique())

        def __str__(self):
            return str(self._storage)

    @staticmethod
    def _col_iter(table, col):
        for i in range(len(table)):
            try:
                yield table[i][col]
            except IndexError:
                yield None

    def __init__(self, table):
        if type(table) != list or any(map(lambda x: type(x) != list, table)):
            raise TypeError('initializer must be a list of lists')

        col_num = max(map(len, table), default=0)
        self._columns = []
        for i in range(col_num):
            self._columns.append(Table._Column(Table._col_iter(table, i)))

        self._names = {}

    def __len__(self):
        return len(self._columns)

    def set_name(self, index, name):
        if name in self._names:
            raise ValueError(f'"{name}" column already exists')

        old_name = self._columns[index].name
        self._columns[index].name = name

        if old_name is not None:
            del self._names[old_name]
        if name is not None:
            self._names[name] = index

    def __getitem__(self, item):
        if type(item) == str:
            return self._columns[self._names[item]]
        elif type(item) == int:
            return self._columns[item]

    def head(self, count):
        res = Table([])

        for col in self._columns:
            res._columns.append(Table._Column(col[:count], name=col.name))

        res._names = self._names
        return res

    def tail(self, count):
        res = Table([])

        for col in self._columns:
            res._columns.append(Table._Column(col[-count:], name=col.name))

        res._names = self._names
        return res

    @staticmethod
    def read(file, titles=False, sep=','):
        names = []
        if titles:
            names = file.readline().split(sep)
        data = []
        for line in file:
            data.append(line.split(sep))

        res = Table(data)

        if titles:
            for i in range(min(len(res), len(names))):
                res.set_name(i, names[i])

    def unique(self):
        res = set()
        for col in self._columns:
            res |= col.unique()
        return res

    def count(self):
        return len(self.unique())

    def __str__(self):
        res = []
        for i in range(len(self)):
            out = [f'Column #{i} ', ': ', str(self._columns[i])]
            name = self._columns[i].name
            if name is not None:
                out.insert(1, f'({name})')
            res.append(''.join(out))
        return '\n'.join(res)



In [26]:
t = Table([[1, 'one', 'один'], [2, 'two'], [3, 'three', 'три']])
print(t)

Column #0 : [1, 2, 3]
Column #1 : ['one', 'two', 'three']
Column #2 : ['один', None, 'три']


In [27]:
t.set_name(0, 'int')
print(t)

Column #0 (int): [1, 2, 3]
Column #1 : ['one', 'two', 'three']
Column #2 : ['один', None, 'три']


In [28]:
t.set_name(2, 'rus')
print(t)

Column #0 (int): [1, 2, 3]
Column #1 : ['one', 'two', 'three']
Column #2 (rus): ['один', None, 'три']


In [29]:
t['rus'][1] = 'два'
print(t)

Column #0 (int): [1, 2, 3]
Column #1 : ['one', 'two', 'three']
Column #2 (rus): ['один', 'два', 'три']


In [30]:
print(t.head(2))

Column #0 (int): [1, 2]
Column #1 : ['one', 'two']
Column #2 (rus): ['один', 'два']


In [31]:
print(t.tail(1))

Column #0 (int): [3]
Column #1 : ['three']
Column #2 (rus): ['три']


In [32]:
t[1].unique()

{'one', 'three', 'two'}

In [33]:
t[0].count()

3

In [34]:
t.unique()

{1, 2, 3, 'one', 'three', 'two', 'два', 'один', 'три'}

In [35]:
t.count()

9

In [38]:
t['rus'][2] = 'один'
t[2].unique(), t[2].count()

({'два', 'один'}, 2)

In [39]:
t.unique(), t.count()

({1, 2, 3, 'one', 'three', 'two', 'два', 'один'}, 8)